In [12]:
import random
from kaggle_environments import make, evaluate
import numpy as np

Добавляем агентов

In [13]:
#Только камень
def rock_agent(observation, configuration):
    return 0

In [14]:
#Только бумага
def paper_agent(observation, configuration):
    return 1

In [15]:
#Только ножницы
def scissors_agent(observation, configuration):
    return 2

In [16]:
#Повторение за оппонентом
def copy_opponent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return random.randrange(0, configuration.signs)

In [17]:
#Случайное действие   
def random_agent(observation, configuration):
    return random.randint(0, configuration.signs - 1)

In [18]:
#Предсказание хода соперника
def counter_opponent(observation, configuration):
    if observation.step > 0:
        return (observation.lastOpponentAction + 1) % configuration.signs
    else:
        return random.randrange(0, configuration.signs)

In [19]:
#Анализ предыдущих ходов соперника
def pattern_matching_agent(observation, configuration, history = []):
    if observation.step > 0:
        history.append(observation.lastOpponentAction)
    if len(history) > 3:
        pattern = history[-3: ]
        prediction = max(set(pattern), key = pattern.count)
        return (prediction + 1) % configuration.signs
    else:
        return random.randrange(0, configuration.signs)

In [20]:
#Копирование выигрышных ходов соперника
def tit_for_tat(observation, configuration):
    if observation.step == 0:
        return random.randrange(0, configuration.signs - 1)
    else:
        return observation.lastOpponentAction

In [21]:
#Использование неэффективной стратегии
def anti_pattern_agent(observation, configuration, history=[]):
    if observation.step > 0:
        history.append(observation.lastOpponentAction)
    if len(history) > 3:
        prediction = max(set(history), key = history.count)
        return (prediction + 2) % configuration.signs
    else:
        random.randint(0, configuration.signs - 1)

In [22]:
#Случайные значения с весом
def weighted_random_agent(observation, configuration):
    weights = [0.4, 0.3, 0.3]
    return random.choices(range(configuration.signs), weights)[0]

In [23]:
#Адапптация под победные ходы соперника
def reinforcement_learning_agent(observation, configuration, score=[0, 0, 0]):
    if observation.step > 0:
        score[observation.lastOpponentAction] += 1
    return (score.index(max(score)) + 1) % configuration.signs

In [24]:
#Ходы по порядку
def cycle_agent(observation, configuration):
    return observation.step % configuration.signs

In [25]:
#Противоположный выбор предыдущему ходу соперника
def mirror_agent(observation, configuration):
    if observation.step == 0:
        return random.randint(0, configuration.signs - 1)
    else:
        return (observation.lastOpponentAction + 2) % configuration.signs

In [26]:
#Добавляем всех агентов в словарь
agents = {
    "Rock Agent": rock_agent,
    "Paper Agent": paper_agent,
    "Scissors Agent": scissors_agent,
    "Copy Opponent": copy_opponent,
    "Random Agent": random_agent,
    "Conter Opponent": counter_opponent,
    "Patetrn Matching": pattern_matching_agent,
    "Tit for tat": tit_for_tat,
    "Anti Pattern": anti_pattern_agent,
    "Weighted Random": weighted_random_agent,
    "Reinforcement Learning": reinforcement_learning_agent,
    "Cycle": cycle_agent,
    "Mirror": mirror_agent
}

In [ ]:
scores = {agent: 0 for agent in agents}
results = {}

#Турнир между агентами
for agent_name_1, agent_1 in agents.items():
    for agent_name_2, agent_2 in agents.items():
        if agent_name_1 != agent_name_2:
            score = evaluate(
                "rps",
                [agent_1, agent_2],
                configuration={"episodeSteps": 1000}
            )
            # Средний результат первой стратегии против второй
            average_score = sum([s[0] for s in score]) / len(score)
            results[(agent_name_1, agent_name_2)] = average_score
            #Определение победителя
            if average_score > 0:
                scores[agent_name_1] += 1  # Победа agent1
            elif average_score < 0:
                scores[agent_name_2] += 1  # Победа agent2
            else:
                scores[agent_name_1] += 0.5  # Ничья
                scores[agent_name_2] += 0.5

#Сортировка агентов по количеству очков
sortred_results = sorted(scores.items(), key = lambda x: x[1], reverse=True)
print("Results:")
for agent, score in sortred_results:
    print(f'{agent}: {score} points')

#Определение максимального количества очков
max_value = max(scores.values())
max_keys = {key for key, value in scores.items() if value == max_value}
#Вывод лучшей(-их) стратегии(-й)
for key in max_keys:
    print(f"The best strategy is: {key}")

Results:
Patetrn Matching: 19.5 points
Conter Opponent: 18.0 points
Cycle: 12.5 points
Anti Pattern: 12.0 points
Reinforcement Learning: 12.0 points
Rock Agent: 11.5 points
Tit for tat: 11.5 points
Paper Agent: 11.0 points
Scissors Agent: 11.0 points
Weighted Random: 11.0 points
Copy Opponent: 9.5 points
Random Agent: 9.5 points
Mirror: 7.0 points
The best strategy is: Patetrn Matching
